In [13]:
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
import numpy as np
import pylab as plt
import tensorflow as tf
from sklearn.preprocessing import normalize
import os,glob

In [86]:
#input_file = '/Users/mylesfoley/git/4thYearProject/0000.txt'

image_list = glob.glob(os.path.join(folder_path, '*.txt'))

data = []
for input_file in file_list:
    input_data = {}
    with open(input_file, 'r') as data_stream:
        for line in data_stream:
            current_line = line.split(' ')
            if not (('DontCare' in current_line[2]) or ('Pedestrian' in current_line[2]) or 
                    ('Cyclist' in current_line[2])):
                if (current_line[2]+ current_line[1]) in input_data.keys():
                    #key by object and unique identifier e.g. Car1
                    input_data[current_line[2] + current_line[1]].append({'frame': current_line[0], 
                         'bbox': current_line[6:10]})
                else:
                    input_data[current_line[2] + current_line[1]] = [{'frame': current_line[0], 
                         'bbox': current_line[6:10]}]
    if len(input_data.keys()) > 27:
        data.append(input_data)

In [83]:
data

[{'Car0': [{'frame': '0',
    'bbox': ['419.230109', '189.412920', '555.229894', '288.627081']},
   {'frame': '1',
    'bbox': ['400.181756', '189.919931', '550.205950', '298.665701']},
   {'frame': '2',
    'bbox': ['377.181832', '190.500489', '544.452131', '310.790502']},
   {'frame': '3',
    'bbox': ['348.858243', '191.171849', '537.796999', '325.727206']},
   {'frame': '4',
    'bbox': ['313.120489', '191.957118', '530.010867', '344.582302']},
   {'frame': '5',
    'bbox': ['266.618745', '192.887972', '520.778763', '369.130439']},
   {'frame': '6',
    'bbox': ['203.646656', '196.402848', '509.678629', '374.000000']},
   {'frame': '7',
    'bbox': ['116.277473', '197.099732', '497.190074', '374.000000']},
   {'frame': '8',
    'bbox': ['0.000000', '197.970612', '481.577000', '374.000000']},
   {'frame': '9',
    'bbox': ['0.000000', '199.089949', '461.499164', '374.000000']},
   {'frame': '10',
    'bbox': ['0.000000', '200.581758', '434.597492', '374.000000']},
   {'frame': '11',

In [87]:
np.array(data).shape
print(len(data))
for item in data:
    print(len(item))

9
61
32
35
55
95
36
134
28
88


In [166]:
sequence_list = []
for sequence in data:
    frame_dict = {}
    for unique_object in sequence:
        
        #print(unique_object)
        #print(len(sequence[unique_object]))
        #check number of frames an object appears in
        if len(sequence[unique_object]) > 28:
            for frame in sequence[unique_object]:
                if frame['frame'] in frame_dict.keys():
                    frame_dict[frame['frame']].append(frame['bbox'])
                else:
                    frame_dict[frame['frame']] = [frame['bbox']]
                #print(frame)
    frame_sample_dict = {}
    for frame in frame_dict:
        if len(frame_dict[frame]) > 3:
            if len(frame_sample_dict) < 100:
                frame_sample_dict[frame] = frame_dict[frame][:3]
    if len(frame_sample_dict) == 100:
        sequence_list.append(frame_sample_dict)
            
            #print(sequence[unique_object])
               
        

In [167]:
for item in sequence_list:
    print(len(item))

100
100
100
100
100
100
100
100


In [171]:
(sequence_list)

[{'40': [['214.891294', '187.925744', '313.374282', '251.932292'],
   ['178.626892', '188.296463', '251.563237', '236.716086'],
   ['148.144090', '190.589347', '212.808807', '233.548608']],
  '41': [['257.762790', '187.337378', '355.082223', '251.338767'],
   ['226.273342', '188.133804', '294.695845', '236.162560'],
   ['196.885603', '189.218924', '259.519140', '231.295183']],
  '42': [['300.405906', '186.750736', '396.997844', '250.732087'],
   ['272.745619', '187.976557', '337.132534', '235.599357'],
   ['243.613667', '188.821843', '302.606450', '230.573617']],
  '43': [['341.350587', '186.243168', '434.008174', '250.519762'],
   ['314.914734', '187.972656', '376.619881', '235.590529'],
   ['289.323795', '188.433951', '345.098159', '229.848810']],
  '44': [['382.308679', '185.735486', '471.468160', '250.265976'],
   ['356.737214', '187.970658', '416.095992', '235.564938'],
   ['334.061069', '188.054846', '386.995202', '229.121214']],
  '45': [['416.358461', '185.227578', '505.054184'

In [193]:
X = []
y = []
list_lengths = []
for sequence in sequence_list:
    sequence_of_frames = []
    for frame in sequence:
        #print(sequence[frame])
        frame_list = []
        for tuples in sequence[frame]:
            #print(tuples)
            frame_list.append(tuples[0])
            frame_list.append(tuples[1])
            frame_list.append(tuples[2])
            frame_list.append(tuples[3])
        #print('***')
        #print(frame_list)
        #print('***')
        sequence_of_frames.append(frame_list)
    X.append(sequence_of_frames)
    y.append(sequence_of_frames)
    #print(sequence.keys())
X = np.array(X)
y = np.array(y)

In [194]:
X.shape

(8, 100, 12)

In [195]:
X

array([[['214.891294', '187.925744', '313.374282', ..., '190.589347',
         '212.808807', '233.548608'],
        ['257.762790', '187.337378', '355.082223', ..., '189.218924',
         '259.519140', '231.295183'],
        ['300.405906', '186.750736', '396.997844', ..., '188.821843',
         '302.606450', '230.573617'],
        ...,
        ['641.359626', '133.130142', '713.298137', ..., '174.627745',
         '734.984262', '234.880995'],
        ['643.084064', '131.041203', '718.181033', ..., '175.210475',
         '743.414408', '238.762988'],
        ['645.639372', '128.779309', '724.896416', ..., '175.860943',
         '752.825293', '243.094661']],

       [['572.007412', '170.281152', '615.244075', ..., '160.957809',
         '338.999575', '186.676063'],
        ['571.771233', '170.031806', '615.307461', ..., '160.719209',
         '333.432385', '187.059701'],
        ['571.531768', '169.778972', '615.371727', ..., '159.094482',
         '327.625446', '186.084846'],
        ...,


In [196]:
Xnorm = []
ynorm = []
for sample in range(0,X.shape[0]):
    Xnorm.append(normalize(X[sample,:], axis=0)[:99])
    
    ynorm.append(normalize(y[sample, :], axis=0)[1:100])

In [216]:
ytr = np.array(ynorm[:6])
Xtr = np.array(Xnorm[:6])
yte = np.array(ynorm[7:])
Xte = np.array(Xnorm[7:])

In [217]:
yte.shape

(1, 99, 12)

In [218]:
modelItem = Sequential()
modelItem.add(LSTM(50, return_sequences = True,input_shape=(99, 12), activation='relu'))
modelItem.add(Dropout(0.2))

modelItem.add(LSTM(units = 50, return_sequences = True, activation='relu'))
modelItem.add(Dropout(0.2))

#modelItem.add(LSTM(units = 50, return_sequences = True, activation='relu'))
#modelItem.add(Dropout(0.2))


modelItem.add(Dense(12))
modelItem.compile(optimizer='adam', loss='mse')

In [219]:
history = modelItem.fit(Xtr, ytr, epochs=500, validation_split=0.2, verbose=1)

Train on 4 samples, validate on 2 samples
Epoch 1/500
4/4 [==============================] - 4s 910ms/step - loss: 0.0121 - val_loss: 0.0109
Epoch 2/500
4/4 [==============================] - 0s 47ms/step - loss: 0.0110 - val_loss: 0.0099
Epoch 3/500
4/4 [==============================] - 0s 49ms/step - loss: 0.0100 - val_loss: 0.0091
Epoch 4/500
4/4 [==============================] - 0s 44ms/step - loss: 0.0092 - val_loss: 0.0083
Epoch 5/500
4/4 [==============================] - 0s 48ms/step - loss: 0.0084 - val_loss: 0.0075
Epoch 6/500
4/4 [==============================] - 0s 47ms/step - loss: 0.0076 - val_loss: 0.0067
Epoch 7/500
4/4 [==============================] - 0s 45ms/step - loss: 0.0069 - val_loss: 0.0061
Epoch 8/500
4/4 [==============================] - 0s 49ms/step - loss: 0.0063 - val_loss: 0.0055
Epoch 9/500
4/4 [==============================] - 0s 46ms/step - loss: 0.0059 - val_loss: 0.0050
Epoch 10/500
4/4 [==============================] - 0s 48ms/step - loss: 0.

Epoch 82/500
4/4 [==============================] - 0s 43ms/step - loss: 0.0012 - val_loss: 7.2305e-04
Epoch 83/500
4/4 [==============================] - 0s 41ms/step - loss: 0.0012 - val_loss: 7.2142e-04
Epoch 84/500
4/4 [==============================] - 0s 42ms/step - loss: 0.0012 - val_loss: 7.1927e-04
Epoch 85/500
4/4 [==============================] - 0s 45ms/step - loss: 0.0012 - val_loss: 7.1715e-04
Epoch 86/500
4/4 [==============================] - 0s 46ms/step - loss: 0.0013 - val_loss: 7.1501e-04
Epoch 87/500
4/4 [==============================] - 0s 46ms/step - loss: 0.0013 - val_loss: 7.1257e-04
Epoch 88/500
4/4 [==============================] - 0s 45ms/step - loss: 0.0013 - val_loss: 7.1038e-04
Epoch 89/500
4/4 [==============================] - 0s 45ms/step - loss: 0.0013 - val_loss: 7.0812e-04
Epoch 90/500
4/4 [==============================] - 0s 46ms/step - loss: 0.0013 - val_loss: 7.0592e-04
Epoch 91/500
4/4 [==============================] - 0s 46ms/step - loss: 

Epoch 161/500
4/4 [==============================] - 0s 46ms/step - loss: 9.8445e-04 - val_loss: 5.4822e-04
Epoch 162/500
4/4 [==============================] - 0s 46ms/step - loss: 9.6388e-04 - val_loss: 5.4684e-04
Epoch 163/500
4/4 [==============================] - 0s 46ms/step - loss: 9.3481e-04 - val_loss: 5.4558e-04
Epoch 164/500
4/4 [==============================] - 0s 43ms/step - loss: 9.2744e-04 - val_loss: 5.4474e-04
Epoch 165/500
4/4 [==============================] - 0s 46ms/step - loss: 9.4381e-04 - val_loss: 5.4521e-04
Epoch 166/500
4/4 [==============================] - 0s 48ms/step - loss: 9.5569e-04 - val_loss: 5.4713e-04
Epoch 167/500
4/4 [==============================] - 0s 46ms/step - loss: 9.1517e-04 - val_loss: 5.4831e-04
Epoch 168/500
4/4 [==============================] - 0s 44ms/step - loss: 9.4947e-04 - val_loss: 5.4544e-04
Epoch 169/500
4/4 [==============================] - 0s 40ms/step - loss: 9.4890e-04 - val_loss: 5.4057e-04
Epoch 170/500
4/4 [=========

Epoch 237/500
4/4 [==============================] - 0s 46ms/step - loss: 7.4339e-04 - val_loss: 4.4568e-04
Epoch 238/500
4/4 [==============================] - 0s 45ms/step - loss: 7.6609e-04 - val_loss: 4.4571e-04
Epoch 239/500
4/4 [==============================] - 0s 46ms/step - loss: 7.3839e-04 - val_loss: 4.4603e-04
Epoch 240/500
4/4 [==============================] - 0s 43ms/step - loss: 7.6484e-04 - val_loss: 4.4734e-04
Epoch 241/500
4/4 [==============================] - 0s 41ms/step - loss: 7.4030e-04 - val_loss: 4.4819e-04
Epoch 242/500
4/4 [==============================] - 0s 42ms/step - loss: 7.5434e-04 - val_loss: 4.4570e-04
Epoch 243/500
4/4 [==============================] - 0s 44ms/step - loss: 7.3810e-04 - val_loss: 4.4333e-04
Epoch 244/500
4/4 [==============================] - 0s 44ms/step - loss: 7.4096e-04 - val_loss: 4.4162e-04
Epoch 245/500
4/4 [==============================] - 0s 47ms/step - loss: 7.3578e-04 - val_loss: 4.3910e-04
Epoch 246/500
4/4 [=========

Epoch 313/500
4/4 [==============================] - 0s 46ms/step - loss: 6.1598e-04 - val_loss: 4.0232e-04
Epoch 314/500
4/4 [==============================] - 0s 47ms/step - loss: 6.0561e-04 - val_loss: 4.0573e-04
Epoch 315/500
4/4 [==============================] - 0s 46ms/step - loss: 5.9672e-04 - val_loss: 4.0390e-04
Epoch 316/500
4/4 [==============================] - 0s 47ms/step - loss: 6.0673e-04 - val_loss: 4.0341e-04
Epoch 317/500
4/4 [==============================] - 0s 44ms/step - loss: 5.9124e-04 - val_loss: 4.0998e-04
Epoch 318/500
4/4 [==============================] - 0s 44ms/step - loss: 6.3041e-04 - val_loss: 4.0685e-04
Epoch 319/500
4/4 [==============================] - 0s 46ms/step - loss: 6.1602e-04 - val_loss: 4.0220e-04
Epoch 320/500
4/4 [==============================] - 0s 41ms/step - loss: 6.1246e-04 - val_loss: 4.0347e-04
Epoch 321/500
4/4 [==============================] - 0s 38ms/step - loss: 5.9955e-04 - val_loss: 4.0409e-04
Epoch 322/500
4/4 [=========

Epoch 389/500
4/4 [==============================] - 0s 48ms/step - loss: 5.4312e-04 - val_loss: 3.9963e-04
Epoch 390/500
4/4 [==============================] - 0s 45ms/step - loss: 5.2865e-04 - val_loss: 3.9683e-04
Epoch 391/500
4/4 [==============================] - 0s 47ms/step - loss: 5.2365e-04 - val_loss: 3.9842e-04
Epoch 392/500
4/4 [==============================] - 0s 46ms/step - loss: 5.1799e-04 - val_loss: 4.0246e-04
Epoch 393/500
4/4 [==============================] - 0s 47ms/step - loss: 5.2796e-04 - val_loss: 3.9893e-04
Epoch 394/500
4/4 [==============================] - 0s 46ms/step - loss: 5.1519e-04 - val_loss: 3.9475e-04
Epoch 395/500
4/4 [==============================] - 0s 46ms/step - loss: 5.3785e-04 - val_loss: 3.9452e-04
Epoch 396/500
4/4 [==============================] - 0s 47ms/step - loss: 5.5292e-04 - val_loss: 3.9762e-04
Epoch 397/500
4/4 [==============================] - 0s 46ms/step - loss: 5.2499e-04 - val_loss: 4.0233e-04
Epoch 398/500
4/4 [=========

Epoch 465/500
4/4 [==============================] - 0s 47ms/step - loss: 4.7225e-04 - val_loss: 4.1098e-04
Epoch 466/500
4/4 [==============================] - 0s 48ms/step - loss: 4.8141e-04 - val_loss: 4.1414e-04
Epoch 467/500
4/4 [==============================] - 0s 44ms/step - loss: 4.8165e-04 - val_loss: 4.0964e-04
Epoch 468/500
4/4 [==============================] - 0s 42ms/step - loss: 4.8266e-04 - val_loss: 4.1128e-04
Epoch 469/500
4/4 [==============================] - 0s 41ms/step - loss: 4.5854e-04 - val_loss: 4.3035e-04
Epoch 470/500
4/4 [==============================] - 0s 45ms/step - loss: 4.8295e-04 - val_loss: 4.2536e-04
Epoch 471/500
4/4 [==============================] - 0s 46ms/step - loss: 4.8002e-04 - val_loss: 4.0528e-04
Epoch 472/500
4/4 [==============================] - 0s 47ms/step - loss: 4.7608e-04 - val_loss: 4.0280e-04
Epoch 473/500
4/4 [==============================] - 0s 46ms/step - loss: 4.9339e-04 - val_loss: 3.9877e-04
Epoch 474/500
4/4 [=========

In [220]:
modelItem.predict(Xte, verbose=0)

array([[[0.08054284, 0.08474094, 0.08046705, ..., 0.08494454,
         0.05983731, 0.07857182],
        [0.08628828, 0.09397691, 0.08363147, ..., 0.09309886,
         0.06680411, 0.08627609],
        [0.08867875, 0.09903832, 0.08443435, ..., 0.09736875,
         0.07171392, 0.09086115],
        ...,
        [0.08884237, 0.09556538, 0.08973087, ..., 0.09875987,
         0.10701947, 0.08904984],
        [0.0915774 , 0.09581219, 0.0909492 , ..., 0.09845512,
         0.10713263, 0.08773655],
        [0.09455574, 0.09621613, 0.09249391, ..., 0.09829754,
         0.10748108, 0.08667293]]], dtype=float32)

In [221]:
yte

array([[[0.07458111, 0.10786647, 0.0708023 , ..., 0.09967549,
         0.07784425, 0.07783697],
        [0.0754301 , 0.10767257, 0.07172936, ..., 0.09982691,
         0.0786577 , 0.07809002],
        [0.0763148 , 0.10746991, 0.07269826, ..., 0.09998295,
         0.07949342, 0.07835057],
        ...,
        [0.09803352, 0.11097851, 0.10941698, ..., 0.09151947,
         0.15483817, 0.09402236],
        [0.09981784, 0.10970143, 0.11694573, ..., 0.09112225,
         0.15746406, 0.0954728 ],
        [0.10193597, 0.10845781, 0.12010898, ..., 0.0906798 ,
         0.16034384, 0.0970886 ]]])